# Matrix Factorization

Matrix factorization is a model-based collaborative filtering method that decomposes the rating matrix R into two factors, i.e. a user feature matrix and a item feature matrix.

$$\text{Ratings Matrix R} = (N \times M)$$

$$N = \text{number of users}$$

$$M = \text{number of items}$$

Recall that R is often *very large* and either can't fit into memory or will not scale as the total number of users increases.

**Problem:** We need a more efficient way to represent item-user matrix R.

**Goal:** Express matrix R using two smaller matrices *W* and *U*.

$$\hat{R} = WU^T$$

Where $\hat{R}$ is an approximation of R. We want it to be as close to R as possible, i.e. minimize error or loss.

## Matrix Factors

We want *W* and *U* to be relatively small to reduce computational complexity.

$$U^{N \times K} = \text{users matrix}$$

$$W^{M \times K} = \text{items matrix}$$

Typically use $K \in [10,50]$, but this is a hyperparameter that should be varied in order to maximize performance.

Note - there are many factors to ratings matrix R. We select a value of K and then optimize W and U in order to reduce total error, where $error = f(R, \hat{R})$.

### Example Reduction in Complexity

Let K = 10, N = 130k, M = 26k

Ratings matrix $R = N \times M = 3.38$ billion.

User matrix $U = N \times 10 = 1.3$ million.

Item matrix $W = M \times 10 = 260$k.

$U + W = 1.56 \text{million}$

**As we can see, working with matrix factors reduces computational complexity by several orders of magnitude.**

$$\frac{1.56 \text{million}}{3.38 \text{billion}} = 0.05\%$$

## Computing Ratings from U and W

Avoid multiplying *U* by *W* directly, the resulting $\hat{R}$ will be very large and consume all resources.

Instead, compute individual scalars at a time and write results to memory.

$$\hat{r}_{ij} = \hat{R}[i,j] = w_i^T u_i$$
$$w_i = W[i], \text{and } u_j = U[j]$$

## Why Does This Work

Singular value decomposition (SVD) tells us that a matrix *X* can be decomposed into 3 separate matrices multiplied together.

$$X = USV^T$$

If we can decompose a matrix into 3 matrices, then of course we can also decompose a matrix into 2 matrices. Just multiply $U \times S$.

![SVD breakdown](svd-breakdown.png)

## Interpreting Matrix Factors

We can treat the K columns of *U* and *W* as features. These features are learned during model optimization.

When we take the dot product $w_i^T u_j$, we get something similar to the cosine similarity. This tells us how well a user's attributes correlate with the item's attributes.

$$
w_i^T u_j = ||w_i|| \text{ } ||u_j|| cos\theta \propto sim(i,j)
$$

In the example below, we have K = 2 and we are *pretending* that the K features represent *comedy* and *action*.

We predict the ratings of $\hat{R}$ by taking the dot product.

![Matrix Factors Visual](matrix-factors-ex.png)

## Learned Features

When we factor R into U and W, U and W become user and item feature matrices, respectively.

**We don't actually know the real meaning of our learned K features.**

These features are latent features, and K is the latent dimensionality. These are also known as "hidden causes"

If we want to get insight on what the features represent, we can cluster users on a single feature and observe which users or items form groups.

Matrix Factorization has automatically found the optimal features for us by learning to predict $\hat{R}$ s.t. error is minimized. 

We don't have to perform manual feature engineering, and performance is usually better.